<h2>개인 구글 드라이브와 colab 연동</h2>

In [1]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [6]:
!pip install transformers
!pip install sentencepiece

root_dir = "/gdrive/My Drive/9-2. Semantic Analysis"

import sys
sys.path.append(root_dir)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<h2>WSD 모델</h2>

In [7]:
from transformers import BertPreTrainedModel, BertModel


class WSD(BertPreTrainedModel):

    def __init__(self, config):
        super(WSD, self).__init__(config) #생성자 오버라이딩딩

        # BERT 모델
        self.bert = BertModel(config)
#training X, 입력해서 뽑아내기만 할거야 
    def forward(self, input_ids, attention_mask): #self-attention 범위 지정을 위한 마스크 like 512 -> 더 크면 받을 필요 없잖아아
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        #outputs의 전체 D는 input 만큼큼

        # BERT 출력 (batch_size, max_length, hidden_size)
        bert_output = outputs[0] #맨 마지막 층 -> batchsiz,max,hidden만큼 나온다다

        return bert_output #전체를 넘긴다다

        #BERT 토크나이저 사용 _ 는 어절의 시작이라는 의미! 옆에 숫자 -> 토큰 인덱스 -> 첫줄 배는 3! 배가 있는 위치 -> 중요할수도도

<h2>데이터 읽고 전처리 하기</h2>

<pre>
<b>1. read_data(file_path)</b>
  "datas.txt" 파일을 읽기 위한 함수
  
  데이터 예시)
    토큰들로 구성된 토큰 시퀀스 \t "배"에 대응하는 토큰 인덱스

    ▁보기 만 ▁해도 ▁배 가 ▁부르 다 .	3
    ▁점 심을 ▁먹 지 ▁못해 ▁배 가 ▁많이 ▁고 팠 다 .	5
    ▁배 ▁한 ▁ 척 이 ▁바다 ▁한 가 운 데 ▁떠 ▁있다 .	0
    ▁그 ▁섬 에는 ▁하루 에 ▁두 ▁번 씩 ▁배 가 ▁들어 온 다 .	8
    ▁나는 ▁ 과 일 ▁중 에서 ▁배 를 ▁가장 ▁좋아 한다 .	6
    ▁사 각 사 각 ▁ 씹 히 는 ▁배 의 ▁맛 이 ▁달 고 ▁시 원 하다 .	8
  
  read_data(file_path)
  args
    file_path : 읽고자 하는 데이터의 경로
  return
    토큰 sequence, "배"에 대응하는 토큰 인덱스를 담고 있는 리스트
    
    출력 예시)
      datas = [ (['▁보기', '만', '▁해도', '▁배', '가', '▁부르', '다', '.'], 3)

                (...),
        
              ]
      
<b>2. convert_data2feature(datas, max_length, tokenizer)</b>
  입력 데이터를 indexing 한 후, padding 추가
  Tensor로 변환
   
  convert_data2feature(datas, max_length, tokenizer)
  args
    datas : 토큰 sequence, "배"에 대응하는 토큰 인덱스를 담고 있는 리스트
    max_length : 입력의 최대 길이
    tokenizer : BERT 토크나이저
  return
    input_ids_features : 입력 문장에 대한 index sequence
    attention_mask_features : padding을 제외한 나머지 실제 토큰 정보를 갖고 있는 sequence
    
  전처리 예시)
    tokens : ['[CLS]', '▁보기', '만', '▁해도', '▁배', '가', '▁부르', '다', '.', '[SEP]']
    input_ids : [2, 2362, 6150, 5002, 2287, 5330, 2432, 5782, 54, 3, ...]
    attention_mask : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...]
 </pre>


In [8]:
import torch


def read_data(file_path):
    with open(file_path, "r", encoding="utf8") as inFile:
        lines = inFile.readlines()

    datas = []
    for line in lines:
        # 입력 데이터를 \t을 기준으로 분리
        pieces = line.strip().split("\t")

        # 입력 토큰 시퀀스, 목표 토큰 인덱스 -> 중요!
        token_sequence, target_token_index = pieces[0].split(" "), int(pieces[1])

        datas.append((token_sequence, target_token_index))
    return datas


def convert_data2feature(datas, max_length, tokenizer):
    input_ids_features, attention_mask_features = [], []

    for token_sequence, target_token_index in datas:

        # CLS, SEP 토큰 추가 [CLS] [token_seq] [SEP]
        tokens = [tokenizer.cls_token]
        tokens += token_sequence
        tokens = tokens[:max_length - 1] #입력이 512보다 크면 잘라야해 -> 512보다 1 작게 잘라 => 뒤에 sep 토큰 넣어줘야 하잖아
        tokens += [tokenizer.sep_token]

        # word piece들을 대응하는 index로 치환
        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # padding을 제외한 실제 데이터 정보를 반영해주기 위한 attention mask
        attention_mask = [1] * len(input_ids)

        # padding 생성
        padding = [tokenizer._convert_token_to_id(tokenizer.pad_token)] * (max_length - len(input_ids))
        padding_for_mask = [0] * (max_length - len(input_ids))

        # padding 추가
        input_ids += padding
        attention_mask += padding_for_mask

        # 변환한 데이터를 각 리스트에 저장
        input_ids_features.append(input_ids)
        attention_mask_features.append(attention_mask)

    # 변환한 데이터를 Tensor 객체에 담아 반환
    input_ids_features = torch.tensor(input_ids_features, dtype=torch.long)
    attention_mask_features = torch.tensor(attention_mask_features, dtype=torch.long)

    return input_ids_features, attention_mask_features

    #tokens : [CLS] '_보기','_만'
    #input_ids : [2,252
    # ????

<h2>WSD 모델을 사용하여 문맥에 따라 변하는 단어 벡터 확인</h2>

<pre>
<b>1. read_data(file_path) 함수를 사용하여 입력 데이터 읽기</b>

<b>2. convert_data2feature(datas, max_length, tokenizer) 함수를 사용하여 데이터 전처리</b>

<b>3. WSD 모델을 활용하여 "배"에 대응하는 벡터 표현 추출</b>

<b>4. 서로 다른 문장에서 사용된 "배"에 대응하는 벡터 표현 사이의 유사도 비교</b>
</pre>

In [9]:
import torch
import operator
import numpy as np

from transformers import BertConfig
from tokenization_kobert import KoBertTokenizer

def get_cos_sim(vector_1, vector_2): #cos sim
  return np.dot(vector_1, vector_2)/(np.linalg.norm(vector_1)*np.linalg.norm(vector_2))


def test(config):
    # BERT config 객체 생성
    bert_config = BertConfig.from_pretrained(pretrained_model_name_or_path=config["pretrained_model_name_or_path"],
                                             cache_dir=config["cache_dir_path"])

    # BERT tokenizer 객체 생성
    bert_tokenizer = KoBertTokenizer.from_pretrained(pretrained_model_name_or_path=config["pretrained_model_name_or_path"],
                                                     cache_dir=config["cache_dir_path"])

    # 데이터 읽기
    datas = read_data(file_path=config["data_path"])

    # 데이터 전처리
    input_ids_features, attention_mask_features = convert_data2feature(datas=datas,
                                                                       max_length=config["max_length"],
                                                                       tokenizer=bert_tokenizer)

    # 사전 학습된 BERT 모델 파일로부터 가중치 불러옴
    model = WSD.from_pretrained(pretrained_model_name_or_path=config["pretrained_model_name_or_path"],
                                cache_dir=config["cache_dir_path"], config=bert_config).cuda()

    input_ids_features = input_ids_features.cuda()
    attention_mask_features = attention_mask_features.cuda()

    # 모델 예측 결과
    bert_outputs = model(input_ids_features, attention_mask_features)

    #여기 안중요
    input_ids_features = input_ids_features.cpu().detach().numpy().tolist()
    bert_outputs = bert_outputs.cpu().detach().numpy().tolist()

    # batch 단위로 구성되어 있어 반복문을 통해 하나씩 확인
    word2vec = {}
    for batch_index in range(len(bert_outputs)): #batch가 그 batch가 아니라 한줄씩 가져오는거!
        input_tokens = bert_tokenizer.convert_ids_to_tokens(input_ids_features[batch_index])
        bert_output = bert_outputs[batch_index]

        token_sequence, target_token_index = datas[batch_index]

        # 입력 토큰 시퀀스 문장으로 변환
        # ['▁보기', '만', '▁해도', '▁배', '가', '▁부르', '다', '.'] -> 보기만 해도 배가 부르다.
        #0,1,2,3 인데, BERT 입장에서 앞에 CLS 있으니까 1 더해줘줘
        sentence = bert_tokenizer.convert_tokens_to_string(token_sequence)

        # token_sequence 앞에 [CLS] 토큰이 추가되었기 때문에 1 추가
        target_token_index += 1

        target_token = input_tokens[target_token_index]
        # 토큰을 단어로 변경 (_배 -> 배)
        target_word = bert_tokenizer.convert_tokens_to_string([target_token])
        target_vector = bert_output[target_token_index] #여기가 핵심! -> 배의 vector를 가져온 구문문
        # 단어, 단어가 사용된 batch_index, 단어가 사용된 문장
        # 배_1 (보기만 해도 배가 부르다.)
        word2vec["{}_{} ({})".format(target_word, batch_index+1, sentence)] = target_vector
        #key, value dic!

    # "배"에 대응하는 각 벡터 표현들 사이의 유사도 계산
    word_similarity = {}
    for word_1, vector_1 in word2vec.items():

        # word_1과 나머지 단어들 사이의 유사도를 담을 리스트 생성
        word_similarity[word_1] = []
        for word_2, vector_2 in word2vec.items():

            # word1과 word2 사이의 코사인 유사도를 계산한 후, 그 결과를 word_similarity 딕셔너리에 저장
            # word_similarity의 key : word1, value : (word2, 유사도)
            #같은 토큰인 경우 건너뜀
            if word_1 == word_2:
              continue
            #word_1과 word_2 사이의 코사인 유사도 계산산
            cos_sim = get_cos_sim(vector_1=vector_1,vector_2 = vector_2)

            #word_2와 대응하는 유사도를 리스트에 추가
            word_similarity[word_1].append((word_2, cos_sim))
            #배_1 <- [(배_2,123),(배_3,123),,,]

            #########################################
            #                                       #
            #                                       #
            #                                       #
            #                                       #
            #           코드 작성 부분              #
            #                                       #
            #                                       #
            #                                       #
            #                                       #
            #########################################

    print("\n단어1 (단어1이 사용된 문장) vs 단어2 (단어2가 사용된 문장) -> 유사도\n")
    for word in word_similarity.keys():
        #[(w1,s1),(w2,s2),...] -> 중요! Score 값(두 번째꺼) 이용해서 sorting!
        # 유사도를 기준으로 정렬, reverse=True를 통해 내림차순으로 정렬
        word_similarity[word] = sorted(word_similarity[word], key=operator.itemgetter(1), reverse=True)

        for index in range(len(word_similarity[word])):
            print("{} vs {} -> {}".format(word, word_similarity[word][index][0], round(word_similarity[word][index][1], 4)))
        print()

In [10]:
import os


if(__name__=="__main__"):
    cache_dir = os.path.join(root_dir, "cache")
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)

    config = {
        "data_path": os.path.join(root_dir, "datas.txt"),
        "cache_dir_path": cache_dir,
        "pretrained_model_name_or_path": "monologg/kobert",
        "max_length": 30
    }

    test(config=config)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.



단어1 (단어1이 사용된 문장) vs 단어2 (단어2가 사용된 문장) -> 유사도

배_1 (보기만 해도 배가 부르다.) vs 배_2 (점심을 먹지 못해 배가 많이 고팠다.) -> 0.7238
배_1 (보기만 해도 배가 부르다.) vs 배_4 (그 섬에는 하루에 두 번씩 배가 들어온다.) -> 0.6015
배_1 (보기만 해도 배가 부르다.) vs 배_6 (사각사각 씹히는 배의 맛이 달고 시원하다.) -> 0.5815
배_1 (보기만 해도 배가 부르다.) vs 배_5 (나는 과일 중에서 배를 가장 좋아한다.) -> 0.5376
배_1 (보기만 해도 배가 부르다.) vs 배_3 (배 한 척이 바다 한가운데 떠 있다.) -> 0.4839

배_2 (점심을 먹지 못해 배가 많이 고팠다.) vs 배_1 (보기만 해도 배가 부르다.) -> 0.7238
배_2 (점심을 먹지 못해 배가 많이 고팠다.) vs 배_5 (나는 과일 중에서 배를 가장 좋아한다.) -> 0.5074
배_2 (점심을 먹지 못해 배가 많이 고팠다.) vs 배_4 (그 섬에는 하루에 두 번씩 배가 들어온다.) -> 0.5071
배_2 (점심을 먹지 못해 배가 많이 고팠다.) vs 배_6 (사각사각 씹히는 배의 맛이 달고 시원하다.) -> 0.4337
배_2 (점심을 먹지 못해 배가 많이 고팠다.) vs 배_3 (배 한 척이 바다 한가운데 떠 있다.) -> 0.4213

배_3 (배 한 척이 바다 한가운데 떠 있다.) vs 배_4 (그 섬에는 하루에 두 번씩 배가 들어온다.) -> 0.7059
배_3 (배 한 척이 바다 한가운데 떠 있다.) vs 배_6 (사각사각 씹히는 배의 맛이 달고 시원하다.) -> 0.6008
배_3 (배 한 척이 바다 한가운데 떠 있다.) vs 배_5 (나는 과일 중에서 배를 가장 좋아한다.) -> 0.5345
배_3 (배 한 척이 바다 한가운데 떠 있다.) vs 배_1 (보기만 해도 배가 부르다.) -> 0.4839
배_3 (배 한 척이 바다 한가운데 떠 있다.) vs 배_2

이제는 output 전체를 가져올수 있는 방법, 벡터를 가지고 cos sim을 아는 방법 공부! 벡터를 활용하는 기능! -> CLS만 쓰는게 아니고 벡터 전체 가져왔잖아아